## Creating the MongoDB Database

In [1]:
#!pip install pymongo

In [2]:
import pymongo

Start the mongo daemon!

In [3]:
from pymongo import MongoClient
import pandas as pd

In [4]:
import numpy as np

In [5]:
client = MongoClient('localhost', 27017)
db = client.project2 #Create the database

In [6]:
#Create the Group Member collection
collection = db.groupMember

In [7]:
courses = pd.read_csv('courses.csv', encoding = "ISO-8859-1", engine='python') 
fam_members = pd.read_csv('familyMembers.csv', encoding = "ISO-8859-1", engine='python') 
group_members = pd.read_csv('groupMembers.csv', encoding = "ISO-8859-1", engine='python') 
pets = pd.read_csv('pets.csv', encoding = "ISO-8859-1", engine='python') 

In [8]:
group_members['age'] = group_members['age'].apply(lambda x: int(str(x)))
group_members['petCount'] = group_members['petCount'].apply(lambda x: int(str(x)))
group_members = group_members.astype({'inState': 'bool'})

In [9]:
courses['course number'] = courses['course number'].apply(lambda x: int(str(x)))
courses['class capacity'] = courses['class capacity'].apply(lambda x: int(str(x)))

In [10]:
fam_members['age'] = fam_members['age'].apply(lambda x: int(str(x)))
fam_members = fam_members.astype({'enrolledAtUVA': 'bool'})

In [11]:
pets['age'] = pets['age'].apply(lambda x: int(float(str(x))))

In [12]:
def correct_encoding(dictionary):
    """Correct the encoding of python dictionaries so they can be encoded to mongodb
    inputs
    -------
    dictionary : dictionary instance to add as document
    output
    -------
    new : new dictionary with (hopefully) corrected encodings"""

    new = {}
    for key1, val1 in dictionary.items():
        # Nested dictionaries
        if isinstance(val1, dict):
            val1 = correct_encoding(val1)

        if isinstance(val1, np.bool_):
            val1 = bool(val1)

        if isinstance(val1, np.int64):
            val1 = int(val1)

        if isinstance(val1, np.float64):
            val1 = float(val1)

        new[key1] = val1

    return new

### Restructuring data into python dicts

In [13]:
for i in range(len(group_members)):    # loop through group members 
    curMem = group_members.iloc[i]
    #print(curMem)
    cur_dict = {}
    for x in range(len(curMem)): 
       # print(x, curMem.index[x])
        if x!=0 and x < 7:              # name - siblingCount props
            cur_dict[curMem.index[x]] = curMem[x]
        if x == 10 or x >19:           # petCount and major - inState props
            cur_dict[curMem.index[x]] = curMem[x]
        if x == 11 or x == 12:           # pets 
            if not pd.isna(curMem[x]):   # if not null find in the pets df 
                petId = curMem[x]
                curPet = pd.DataFrame(pets.loc[pets['id '] == petId]).reset_index()
                if not cur_dict.get('pets'): 
                    cur_dict['pets'] = {}
               # print(curPet['name'])
                cur_dict['pets'][curMem.index[x]] = {}
                cur_dict['pets'][curMem.index[x]]['name'] = curPet['name'][0]
                cur_dict['pets'][curMem.index[x]]['species'] = curPet['species'][0]
                cur_dict['pets'][curMem.index[x]]['breed'] = curPet['breed'][0]
                cur_dict['pets'][curMem.index[x]]['gender'] = curPet['gender'][0]
                cur_dict['pets'][curMem.index[x]]['age'] = curPet['age'][0]
                cur_dict['pets'][curMem.index[x]]['color'] = curPet['color'][0]
        if x > 12 and x <20:           # courses 
            if not pd.isna(curMem[x]): # if not null, find in courses df 
                courseId = curMem[x]
                curCourse = pd.DataFrame(courses.loc[courses['id'] == courseId])
                if not cur_dict.get('courses'): 
                    cur_dict['courses'] = {}
                #print(curCourse['course mnemonic'][courseId])
                cur_dict['courses'][curMem.index[x]] = {}
                cur_dict['courses'][curMem.index[x]]['mnemonic'] = curCourse['course mnemonic'][courseId]
                cur_dict['courses'][curMem.index[x]]['courseNumber'] = int(curCourse['course number'][courseId])
                cur_dict['courses'][curMem.index[x]]['department'] = curCourse['department'][courseId]
                cur_dict['courses'][curMem.index[x]]['classTitle'] = curCourse['class title'][courseId]
                cur_dict['courses'][curMem.index[x]]['professor'] = curCourse['professor'][courseId]
                cur_dict['courses'][curMem.index[x]]['meetingDay(s)'] = curCourse['meeting day(s)'][courseId]
                cur_dict['courses'][curMem.index[x]]['type'] = curCourse['type'][courseId]
                cur_dict['courses'][curMem.index[x]]['classCapacity'] = int(curCourse['class capacity'][courseId])
                cur_dict['courses'][curMem.index[x]]['courseDescription'] = curCourse['course description'][courseId]
                #print(cur_dict.get('courses'))
    for y in range(len(fam_members)): 
        curFam = fam_members.iloc[y]
        if curFam['groupMemberRelative'] == curMem['id']: 
            if not cur_dict.get('fam_members'): 
                cur_dict['fam_members'] = {}
            cur_dict['fam_members'][curFam['name']] = {}
            cur_dict['fam_members'][curFam['name']]['relation'] = curFam['relation']
            cur_dict['fam_members'][curFam['name']]['age'] = int(curFam['age'])
            cur_dict['fam_members'][curFam['name']]['gender'] = curFam['gender']
            cur_dict['fam_members'][curFam['name']]['birthday'] = curFam['birthday']
            cur_dict['fam_members'][curFam['name']]['favIceCream'] = curFam['favIceCream']
            cur_dict['fam_members'][curFam['name']]['petCount'] = curFam['petCount']
            cur_dict['fam_members'][curFam['name']]['enrolledAtUVA'] = bool((curFam['enrolledAtUVA']))
            
    cur_dict = correct_encoding(cur_dict)
    collection.insert_one(cur_dict)

In [14]:
pets.columns

Index(['id ', 'name', 'species', 'breed', 'gender', 'age', 'ownerGroup',
       'color'],
      dtype='object')

## MongoDB Queries
https://www.geeksforgeeks.org/nested-queries-in-pymongo/

https://pymongo.readthedocs.io/en/stable/examples/aggregation.html

https://www.tutorialspoint.com/mongodb/mongodb_query_document.htm

In [15]:
from bson.son import SON

In [16]:
collection.count_documents({})

5

Which group members are taking classes in the CS department?

In [17]:
for val in collection.find({ "$or": [
         {'courses.course1.department': 'Computer Science'}, 
         {'courses.course2.department':'Computer Science'},
        {'courses.course3.department':'Computer Science'}, 
        {'courses.course4.department':'Computer Science'}, 
         {'courses.course5.department':'Computer Science'},
         {'courses.course6.department':'Computer Science'}, 
        {'courses.course7.department':'Computer Science'}
      ]}): 
    print(val.get('name'))

Anna Grace Calhoun
Daniel Tohti
David Siamon


What is the average age of the group members’ pets?

In [18]:
collection.find({"$avg" :"pets:pet1:age" })

Which group members are over 20?

In [19]:
print(collection.find_one({'name':'Daniel Tohti'}))

{'_id': ObjectId('60b9006d3734963720142e96'), 'name': 'Daniel Tohti', 'birthday': '2001/12/28', 'birthCity': 'Gainesville, FL', 'hometown': 'Fairfax, VA', 'age': 19, 'siblingCount': 2, 'petCount': 1, 'pets': {'pet1': {'name': 'Leo', 'species': 'Dog', 'breed': 'German Shepherd', 'gender': 'M', 'age': 2, 'color': 'Black'}}, 'courses': {'course1': {'mnemonic': 'APMA', 'courseNumber': 3140, 'department': 'Applied Mathematics', 'classTitle': 'Partial Differential Equations', 'professor': 'Diana Morris', 'meetingDay(s)': 'M,W,F', 'type': 'Lecture', 'classCapacity': 300, 'courseDescription': 'Partial differential equations that govern physical phenomena in science and engineering. Separation of variables, superposition, Fourier series, Sturm-Liouville eigenvalue problems, eigenfunction expansion techniques. Particular focus on the heat, wave, and Laplace partial differential equations in rectangular, cylindrical, and spherical coordinates.'}, 'course2': {'mnemonic': 'APMA', 'courseNumber': 35